In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import integrate
from matplotlib import pyplot as plt

In [ ]:
sns.set_context('poster')

#Calculate $N_{\rm \small effective}$ from $N_{\rm \small pole}$ and $N_{\rm \small equator}$.

###$N_{\rm \small effective} = N_{\rm \small pole} \frac{2 \Omega_{\rm \small pole}}{4\pi} + N_{\rm \small equator} \frac{4\pi - 2 \Omega_{\rm \small pole}}{4\pi}$

###$\Omega_{\rm \small pole} = \int_0^{2\pi}{\rm d}\phi \int_0^{\pi/4}{\rm sin}\theta \,{\rm d}\theta  = 2\pi \left( 1 - \frac{1}{\sqrt{2}} \right) \approx 2\pi \times 0.29289 = 1.840282$

#From our [fit](Local Optical Depth Prescription.ipynb),
##$N_{\rm \small pole} = 10^{0.532301 {\rm log_{10}}(n) + 19.636552}$ and $N_{\rm \small equator} = 10^{0.626204 {\rm log_{10}}(n) + 19.573490}$

In [ ]:
def N_p(n):
    exponent = 0.532301 * np.log10(n) + 19.636552
    return 10**exponent

def N_eq(n):
    exponent = 0.626204 * np.log10(n) + 19.573490
    return 10**exponent

In [ ]:
Omega_pole = 1.840282
fourpi = 4 * np.pi
a = 2 * Omega_pole / fourpi
b = (fourpi - 2*Omega_pole)/fourpi
def N_eff(n):
    x = N_p(n)
    y = N_eq(n)
    return a*x + b*y

In [ ]:
def beta(epsilon):
    mHc2 = 938272046 #rest energy of a proton in eV
    return np.sqrt(1 - (epsilon/mHc2 + 1)**(-2))

def f(epsilon):
    B0 = 0.01
    B = beta(epsilon)
    return (1 + 0.0185 * np.log(B)) * 2*B**2 / (B0**3 + 2*B**3)

def dnCR(epsilon, ucr, emin, emax):
    return ucr / epsilon / epsilon / np.log(emax/emin)

def dedt(n, epsilon):
    return 1.82e-7 * n * f(epsilon)

def Dp(n,epsilon):
    return beta(epsilon) * 2.99e10 * epsilon / dedt(n,epsilon)

def sigma(n, epsilon):
    return 1 / n / Dp(n,epsilon)

def tau(n, epsilon):
    return sigma(n, epsilon) * N_eff(n)

In [ ]:
def integrand(epsilon, n, ucr, emin, emax, attenuation=True):
    if attenuation:
        return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax) * np.exp(-tau(n, epsilon))
    else:
        return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax)

In [ ]:
def logintegrate(func, n, ucr, emin, emax, attenuation=True, n_steps=1000):
    #print 'emin: 1e'+str(np.log10(emin)), 'emax: 1e'+str(np.log10(emax)),
    #print 'n:',n, 'ucr:',ucr, 'tau:',attenuation
    integral = 0.0
    logmin = np.log10(emin)
    logmax = np.log10(emax)
    for i in xrange(n_steps):
        epsilon = (logmax - logmin) / n_steps * (i + 0.5) + logmin
        e_start = (logmax - logmin) / n_steps * (i) + logmin
        e_stop = (logmax - logmin) / n_steps * (i + 1.0) + logmin
        epsilon = 10**epsilon
        e_start = 10**e_start
        e_stop = 10**e_stop
        De = e_stop - e_start
        integral += func(epsilon, n, ucr, emin, emax, attenuation) * De
    return integral

In [ ]:
def oldintegrand(epsilon, n, ucr, emin, emax, attenuation):
    return f(epsilon) / epsilon / epsilon / np.log(emax/emin)
logintegrate(oldintegrand, 1, 1, 1e6, 1e15, False)  * 1.82e-7 / 50

In [ ]:
@np.vectorize
def h_rate(n, ucr, emin, emax, attenuation=True):
    E_heat = 6 / 6.24150934e11 # convert from eV to erg
    return E_heat * logintegrate(integrand, n, ucr, emin, emax, attenuation)

@np.vectorize
def k_rate(n, ucr, emin, emax, attenuation=True):
    return logintegrate(integrand, n, ucr, emin, emax, attenuation) / n

In [ ]:
k_rate(1, 1, 1e6, 1e15, True)

In [ ]:
ucr = 1.
E_min = 1e6
E_max = 1e15
attenuation = True
n = np.logspace(-4,12.2,82)
kh_rates = pd.DataFrame(index=np.linspace(-4,12.2,82))
kh_rates['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
kh_rates['kHe'] = 0
kh_rates['kHep'] = 0
kh_rates['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n 
kh_rates['hHe'] = 0
kh_rates['hHep'] = 0
kh_rates['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
attenuation = False
n = np.logspace(-4,12.2,82)
khr = pd.DataFrame(index=np.linspace(-4,12.2,82))
khr['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
khr['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n
khr['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
kh_rates.head(3)

In [ ]:
kh_rates.tail(3)

In [ ]:
kh_rates['ion'].plot(logy=True)
plt.plot(khr.index, khr.ion, '--', color='grey')
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('Ionization Rate [s$^{-1}$]')
plt.savefig('figures/khrates/ionrate.png', bbox_inches='tight')

In [ ]:
kh_rates['heat'].plot(logy=True)
plt.plot(khr.index, khr.heat, '--', color='grey')
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('Heating Rate / n [erg s$^{-1}$]')
plt.savefig('figures/khrates/hrate.png', bbox_inches='tight')

In [ ]:
kh_rates['nheat'].plot(logy=True)
plt.plot(khr.index, khr.nheat, '--', color='grey')
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('Heating Rate [erg s$^{-1}$ cm$^{-3}$]')
plt.savefig('figures/khrates/nhrate.png', bbox_inches='tight')

###### As currently designed, the optical depth for each individual species is calculated using the total column density.  Thinking I probably need to use the column density specific to each species.

In [ ]:
kh_data = np.column_stack((kh_rates.index.values, kh_rates[['ion', 'kHe', 'kHep', 'heat', 'hHe', 'hHep']].values))

In [ ]:
kh_data.shape

In [ ]:
np.savetxt('kh_rates.dat', kh_data, fmt='%.8e %.8e %.8e %.8e %.8e %.8e %.8e')

In [ ]:
!head kh_rates.dat

In [ ]:
!tail kh_rates.dat